In [1]:
import sys, os, re, json, time

import pandas as pd
import pickle
import h5py

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import plotting
from PIL import Image
from tqdm import tqdm
from utils import imread, img_data_2_mini_batch, imgs2batch

from sklearn import metrics

from attention import Enc, Dec, EncDec
from data_loader import VQADataSet

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision import transforms

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
N = 10000
dataset_filename = "./data/data_{}.pkl".format(N)
dataset = None
print(dataset_filename)
if (os.path.exists(dataset_filename)):
    with open(dataset_filename, 'rb') as handle:
        print("reading from " + dataset_filename)
        dataset = pickle.load(handle)
else:
    dataset = VQADataSet(Q=N)
    with open(dataset_filename, 'wb') as handle:
        print("writing to " + dataset_filename)
        pickle.dump(dataset, handle)

assert(dataset is not None)
def debug(v,q,a):
    print('\nV: {}\nQ: {}\nA: {}'.format(v.shape, q.shape, a.shape))


./data/data_10000.pkl
reading from ./data/data_10000.pkl


In [3]:
embed_size        = 300
hidden_size       = 1024
batch_size        = 50
ques_vocab_size   = len(dataset.vocab['question'])
c                 = len(dataset.vocab['answer'])
num_layers        = 1
n_epochs          = 20
learning_rate     = 0.01
momentum          = 0.98
attention_size    = 512
debug             = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = EncDec(embed_size, hidden_size, attention_size, ques_vocab_size, c, num_layers, debug).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [4]:
c

3891

In [5]:
def eval_model(data_loader, model, criterion, optimizer, batch_size, training=False,
              total_loss_over_epochs=[], scores_over_epochs=[]):
    running_loss = 0.
    final_labels, final_preds = [], []
    if data_loader is None:
        return
    
    if training:
        model.train()
    else:
        model.eval()
    
    for i, minibatch in enumerate(data_loader):
        # extract minibatch
        t0 = time.time()
        idxs, v, q, a, q_len = minibatch
        
        # convert torch's DataLoader output to proper format.
        # torch gives a List[Tensor_1, ... ] where tensor has been transposed. 
        # batchify transposes back.`
        v = v.to(device)
        q = VQADataSet.batchify_questions(q).to(device)
        a = a.to(device)

        logits = model(v, q, q_len)
        
        loss = F.nll_loss(logits, a)
    
        print(loss.item())
        
        if training and optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    
    
    
    


In [6]:

train_loader = dataset.build_data_loader(train=True, args={'batch_size': batch_size})
test_loader  = dataset.build_data_loader(test=True, args={'batch_size': batch_size})

best_score = 0

train_all_loss, train_all_labels, train_all_preds = [], [], []

total_loss_over_epochs, scores_over_epochs = plotting.get_empty_stat_over_n_epoch_dictionaries()

for epoch in tqdm(range(n_epochs)):
    t0= time.time()
    tr_loss, tr_labels, tr_preds = eval_model(data_loader = train_loader,
                                     model       = model,
                                     criterion   = criterion,
                                     optimizer   = optimizer,
                                     batch_size  = batch_size,
                                     training    = True,
                                     total_loss_over_epochs = total_loss_over_epochs,
                                     scores_over_epochs     = scores_over_epochs)
    
#     train_scores = metrics.precision_recall_fscore_support(tr_labels,
#                                                            tr_preds,
#                                                            average='weighted')
    
#     total_loss_over_epochs['train_loss'].append(tr_loss)
#     scores_over_epochs['train_scores'].append(train_scores)
    
#     if True:# or epoch%1 == 0:
#         print("#==#"*5 + "epoch: {}".format(epoch) + "#==#"*5)
#         print("time: {}".format(time.time()-t0))
#         print(train_scores)
#     plotting.plot_score_over_n_epochs(scores_over_epochs, score_type='precision', fig_size=(8,5))
#     plotting.plot_loss_over_n_epochs(total_loss_over_epochs, fig_size=(8, 5), title="Loss")
    
    
    
    

  0%|          | 0/20 [00:00<?, ?it/s]

batch_size: 50 shuffle: True
batch_size: 50 shuffle: False
8.272541999816895
58.45856475830078
104.03276062011719
31.23953628540039
13.028277397155762
10.342129707336426
8.001704216003418
8.199151992797852
136.3775634765625
7.665517807006836
7.716432571411133
7.602004528045654
7.748861312866211
7.285489559173584
7.5139312744140625
7.4652557373046875
7.795978546142578
7.864872932434082
7.552169322967529
7.512897491455078
7.867869853973389
7.368079662322998
7.559175491333008
7.528205394744873
7.679619312286377
7.157935619354248
7.1117730140686035
7.139580726623535
7.871028423309326
7.655363082885742
7.28419303894043
7.3453688621521
7.685708999633789
8.440828323364258
6.81636905670166
8.257708549499512
7.242302894592285
6.717006206512451
7.308776378631592
7.483102321624756
7.362673282623291
6.996409893035889
6.673761367797852
8.49937629699707
6.611283779144287
7.940474033355713
6.9325947761535645
7.046403408050537
7.1791582107543945
7.475282192230225
7.472904205322266
7.952498912811279
7.

TypeError: 'NoneType' object is not iterable

In [ ]:
# print(type(tr_labels))
# print(type(tr_preds))


In [ ]:
# print(tr_labels[0])
# print(tr_preds[0])